<a href="https://colab.research.google.com/github/Tvisha-tech88/Summer-Analytics-2025/blob/main/CapstoneProject_TvishaBansal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Dynamic Pricing for Urban Parking Lots**
       Capstone Poject - Summer Analytics 2025

                   - Tvisha Bansal

# 📌 Introduction
Urban parking spaces are limited and experience highly dynamic demand. Static pricing often leads to inefficiencies like congestion or underutilization.
The goal of this project is to build a dynamic pricing system for urban parking lots based on:

Real-time data

Demand and vehicle types

Traffic congestion

Special events

Competition from nearby parking spaces

We simulate this system using Python, Pandas, Numpy, Pathway, and Bokeh, and design three progressive pricing models.



# 📂 Dataset Description
Dataset: 14 parking spaces, 73 days, 18 time points per day (every 30 minutes from 8:00 AM to 4:30 PM)

Features:

Location: Latitude, Longitude

Parking Lot: Capacity, Occupancy, Queue Length

Vehicle Info: Type of vehicle (car, bike, truck)

Environment: Traffic congestion level, Special day indicator



# 🛠️ Libraries and Data Loading

In [ ]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
import time


output_notebook()

from google.colab import files


uploaded = files.upload()

# Load Dataset
df = pd.read_csv('dataset.csv')


Saving dataset.csv to dataset.csv


# 🔍 Data Preprocessing

In [ ]:
# Feature Engineering
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']

# Assign weights to vehicle types
df['VehicleTypeWeight'] = df['VehicleType'].map({'car': 1.0, 'bike': 0.5, 'truck': 1.5})

# Fill missing values
df.fillna(0, inplace=True)


# 🧮 Model 1: Baseline Linear Model

In [ ]:
# Parameters
base_price = 10
alpha = 2

# Store results
prices_baseline = []

# Initial price
previous_price = base_price

# Baseline Pricing Model
def baseline_linear_model(occupancy_rate, previous_price):
    return previous_price + alpha * occupancy_rate


# 📊 Define Pricing Models

In [ ]:
# Pricing Parameters
base_price = 10
alpha = 2
beta = 1.5
gamma = 1.2
delta = 2.0
epsilon = 1.0
lmbda = 0.05

# Initialize previous price
previous_price = base_price

# Store calculated prices
prices_baseline = []
prices_demand = []

# Model 1: Baseline Linear Model
def baseline_linear_model(occupancy_rate, previous_price):
    return previous_price + alpha * occupancy_rate

# Model 2: Demand-Based Pricing Model
def demand_based_model(row):
    demand = (alpha * row['OccupancyRate'] +
              beta * row['QueueLength'] -
              gamma * row['TrafficConditionNearby'] +  # Correct column used
              delta * row['IsSpecialDay'] +
              epsilon * row['VehicleTypeWeight'])

    # Normalize demand
    normalized_demand = (demand - df['OccupancyRate'].min()) / (df['OccupancyRate'].max() - df['OccupancyRate'].min())
    price = base_price * (1 + lmbda * normalized_demand)

    # Ensure price stays within 0.5x to 2x base price
    return max(min(price, 2 * base_price), 0.5 * base_price)


# 📈 Real-Time Visualization Setup

In [ ]:
# Bokeh Data Source
source = ColumnDataSource(data=dict(time=[], price=[]))

# Create Bokeh Figure
p = figure(title="Real-Time Parking Price (Demand Model)", x_axis_label='Time Step', y_axis_label='Price ($)', height=350)
p.line(x='time', y='price', source=source)

# Display the plot
handle = show(p, notebook_handle=True)




# 🧮 Model 2: Demand-Based Pricing Model

In [ ]:
# Parameters
beta = 1.5
gamma = 1.2
delta = 2.0
epsilon = 1.0
lmbda = 0.05

prices_demand = []

# Demand Model
def demand_based_model(row):
    demand = (alpha * row['OccupancyRate'] +
              beta * row['QueueLength'] -
              gamma * row['Traffic'] +
              delta * row['SpecialDay'] +
              epsilon * row['VehicleTypeWeight'])

    # Normalize demand to keep price within reasonable limits
    normalized_demand = (demand - df['OccupancyRate'].min()) / (df['OccupancyRate'].max() - df['OccupancyRate'].min())
    price = base_price * (1 + lmbda * normalized_demand)

    # Bound the price between 0.5x and 2x the base price
    return max(min(price, 2 * base_price), 0.5 * base_price)


# 🔄 Real-Time Simulation (Models 1 and 2)

In [ ]:
# Map traffic levels to numbers
traffic_mapping = {'High': 3, 'Medium': 2, 'Low': 1}
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map(traffic_mapping)

# Fill any missing values
df.fillna(0, inplace=True)

for idx, row in df.iterrows():
    # Model 1: Baseline Linear Model
    price_baseline = baseline_linear_model(row['OccupancyRate'], previous_price)
    prices_baseline.append(price_baseline)
    previous_price = price_baseline

    # Model 2: Demand-Based Pricing Model
    try:
        price_demand = demand_based_model(row)
    except Exception as e:
        print(f"Error at index {idx}: {e}")
        continue  # Skip problematic rows

    prices_demand.append(price_demand)

    # Real-Time Visualization (Demand Model)
    source.stream({'time': [idx], 'price': [price_demand]})
    push_notebook(handle=handle)
    time.sleep(0.01)  # Simulate streaming delay

print("✅ Real-Time Pricing Simulation Completed!")






✅ Real-Time Pricing Simulation Completed!


# 📚 Key Assumptions
Base price = $10

Price range: $(5 - 20)

Vehicle weights: Car = 1.0, Bike = 0.5, Truck = 1.5

Real-time simulation approximated with a small time delay



# 🎯 Conclusion
This project successfully demonstrates a dynamic, real-time pricing model for urban parking lots using real-time simulation in Python and Bokeh visualization.
It can be further enhanced by:

Adding competitive pricing based on proximity

Connecting to live traffic/event APIs

Deploying as a real-time dashboard using Pathway